In [1]:
import pandas as pd
import re
from nltk import word_tokenize
import string
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import pickle 

In [2]:
import os
os.getcwd()

'C:\\Users\\kumrajni\\AppData\\Local\\Continuum\\anaconda3\\Scripts\\Jupyter_Learnign\\Jupyter_Learnign'

In [5]:
data = pd.read_csv('Paycd.csv',header=0,encoding = "ISO-8859-1")

In [4]:
data['newcol'] = data['Paycd']+data['Category']
data.head()

,Paycd,Category,newcol
0,$Bonus,Premium Time,$BonusPremium Time
1,$Tips,Premium Time,$TipsPremium Time
2,3rd Shift Diff,Absence,3rd Shift DiffAbsence
3,CFRA,Premium Time,CFRAPremium Time
4,Jury,Absence,JuryAbsence


In [ ]:
data['newco2'] = data['Paycd'].apply(lambda x:x+data['Category'])

In [ ]:
data

In [135]:
## here i will be trainign data with some value and testiong it with the data set with Others as Category
data_test = data[data['Category']=='Other']
print(len(data_test))

5702


In [305]:
## as there are these category which doesnt make sense so train model execlusing these
data_train = data[(data['Category']!='Other') & (data['Category']!='Exclude from Analytics') & (data['Category']!='Uncategorized')]
print(len(data_train))

12313


In [306]:
data_train.head()

,Paycd,Category
0,$Bonus,Premium Time
1,$Tips,Premium Time
2,3rd Shift Diff,Absence
3,CFRA,Premium Time
4,Jury,Absence


In [307]:
data_train['Category'].value_counts()

Regular         4692
Absence         4552
Overtime        1870
Premium Time    1199
Name: Category, dtype: int64

In [308]:
## now make a new column and map our labels and then we will drop the original column
data_train['Category_'] = data['Category'].map({'Regular':'0','Absence':'1','Overtime':'2','Premium Time':'3'})
print(data_train['Category_'].value_counts())

0    4692
1    4552
2    1870
3    1199
Name: Category_, dtype: int64


C:\Users\kumrajni\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [309]:
data_train.drop(labels=['Category'],axis=1,inplace=True)

C:\Users\kumrajni\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [6]:
data.head()

,Paycd,Category
0,$Bonus,Premium Time
1,$Tips,Premium Time
2,3rd Shift Diff,Absence
3,CFRA,Premium Time
4,Jury,Absence


In [8]:
X = data['Paycd']
y = data['Category']

In [9]:
print(X.shape) 
print(y.shape) 
print(data['Paycd'].isnull().sum())

(20327,)
(20327,)
0


In [17]:
punc = set(string.punctuation)
ps = PorterStemmer()
def removedollar(text):
    no_doll = re.sub(r'\$','',text)
    no_doll = re.sub(r'\<','',no_doll)
    no_doll = re.sub(r'\>','',no_doll)
    tokenized = word_tokenize(no_doll)
    no_punc = [word for word in tokenized if not word.lower() in punc]
    stemmed = [ps.stem(word) for word in no_punc]
    word_alpa = [word for word in stemmed if word.isalpha()]
    word_alpa = " ".join(word_alpa)
    return word_alpa

In [18]:
data_stem = X.apply(removedollar)

In [23]:
data_stem.ix[14]

'FR bank holiday taken No OT is am are I know and the junk word are so touch'

In [19]:
data_stem.head(20)
## we need to remove '' loke 11 row 

0                                                  bonu
1                                                   tip
2                                            shift diff
3                                                  cfra
4                                                  juri
5                                     short term disabl
6                                           all overtim
7                                                bereav
8                                           worker comp
9                                               regular
10                                                     
11                                               travel
12                                         BK hr tkn OT
13                                          vestsmb hhw
14    FR bank holiday taken No OT is am are I know a...
15                                                     
16                                             oncal OT
17                                              

In [317]:
drop_index = data_stem.index[data_stem.values=='']

In [318]:
data_stem.drop(drop_index,inplace=True)

In [319]:
y.drop(drop_index,inplace=True)

C:\Users\kumrajni\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\generic.py:2184: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(result)


In [320]:
print(data_stem.shape) 
print(y.shape) 

(10956,)
(10956,)


In [321]:
CV = CountVectorizer()
cv_Dat = CV.fit_transform(data_stem)

In [322]:
cv_Dat

<10956x2912 sparse matrix of type '<class 'numpy.int64'>'
	with 25998 stored elements in Compressed Sparse Row format>

In [292]:
DTM_paycd = pd.DataFrame(cv_Dat.toarray(),columns=CV.get_feature_names())

In [293]:
DTM_paycd.head()

,bank,bk,fr,hhw,holiday,hr,no,oncal,ot,taken,tkn,travel,vestsmb
0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,1,0,0,0,1,0,0,1,0,1,0,0
2,0,0,0,1,0,0,0,0,0,0,0,0,1
3,1,0,1,0,1,0,1,0,1,1,0,0,0
4,0,0,0,0,0,0,0,1,1,0,0,0,0


In [338]:
## naIVE BASE classifier
model_logi = MultinomialNB()
model_logi.fit(cv_Dat,y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [324]:
data_test.head()

,Paycd,Category
15,MA5_SHIFT1_RG,Other
17,RI1_SHIFTWE_RG,Other
18,Suspended No Pay,Other
19,WI5_SHIFT3_DT,Other
22,VESTSMB Shift 2 OT Diff,Other


In [325]:
X_test = data_test['Paycd']

In [326]:
test_stem = X_test.apply(removedollar)

In [328]:
## removing index where we have '' as value
test_index_drop = test_stem.index[test_stem.values=='']   ## data_stem.index[data_stem.values=='']
test_stem.drop(test_index_drop,inplace=True)

In [329]:
test_stem.head()

18           suspend No pay
22    vestsmb shift OT diff
23          driver demurrag
33                  shoptim
36               daili rate
Name: Paycd, dtype: object

In [330]:
test_DTM = CV.transform(test_stem)

In [333]:
DTM_paycd = pd.DataFrame(test_DTM.toarray(),columns=CV.get_feature_names())

In [334]:
DTM_paycd.head()

,aa,ab,abay,abb,abc,abctim,abd,abe,abov,absenc,...,zzztrain,zzzunapprov,zzzvacat,zzzyk,zzzzhol,zzzzholiday,zzzznur,zzzzon,zzzzunpaid,zzzzzunpaid
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [335]:
y_pred = model_logi.predict(test_DTM)

In [337]:
y_pred[0:10]

array(['1', '3', '0', '1', '0', '1', '2', '1', '0', '0'], dtype='<U1')

In [342]:
data_test = pd.read_csv("Paycd_test.csv",header=0,encoding="ISO-8859-1")

In [345]:
data_test.head()

,Paycd,Category
0,~DISABLE,Other
1,~FURLOW,Other
2,~HALFOCC,Other
3,~LATE>1H,Other
4,~MANERLY,Other


In [346]:
data_test.Category.value_counts()

Absence                   976
Other                     954
Regular                   671
Overtime                  311
Exclude from Analytics    294
Uncategorized             249
Premium Time              223
Name: Category, dtype: int64

In [348]:
data_test = data_test[(data_test['Category']=='Absence') | (data_test['Category']=='Regular') | (data_test['Category']=='Overtime')]

In [350]:
data_test.Category.value_counts()

Absence     976
Regular     671
Overtime    311
Name: Category, dtype: int64

In [369]:
X_test_wfn = data_test['Paycd']
y_test_wfn = data_test['Category'] 

In [370]:
X_test_wfn = X_test_wfn.apply(removedollar)

In [371]:
print(len(X_test_wfn))

1958


In [372]:
##X_test_wfn.head(20)
index_drop_wfn = X_test_wfn.index[X_test_wfn.values=='']

In [373]:
X_test_wfn.drop(index_drop_wfn,inplace=True)
y_test_wfn.drop(index_drop_wfn,inplace=True)
print(len(X_test_wfn))
print(len(y_test_wfn))

1711
1711


In [386]:
y_test_wfn = y_test_wfn.map({'Regular':'0','Absence':'1','Overtime':'2','Premium Time':'3'})

In [375]:
X_test_wfn.head()

9         adj
10    callvac
11        cat
13      griev
14      lbreg
Name: Paycd, dtype: object

In [377]:
test_DTM_WFN = CV.transform(X_test_wfn)
print(pd.DataFrame(test_DTM_WFN.toarray(),columns=CV.get_feature_names()))

      aa  ab  abay  abb  abc  abctim  abd  abe  abov  absenc     ...       \
0      0   0     0    0    0       0    0    0     0       0     ...        
1      0   0     0    0    0       0    0    0     0       0     ...        
2      0   0     0    0    0       0    0    0     0       0     ...        
3      0   0     0    0    0       0    0    0     0       0     ...        
4      0   0     0    0    0       0    0    0     0       0     ...        
5      0   0     0    0    0       0    0    0     0       0     ...        
6      0   0     0    0    0       0    0    0     0       0     ...        
7      0   0     0    0    0       0    0    0     0       0     ...        
8      0   0     0    0    0       0    0    0     0       0     ...        
9      0   0     0    0    0       0    0    0     0       0     ...        
10     0   0     0    0    0       0    0    0     0       0     ...        
11     0   0     0    0    0       0    0    0     0       0     ...        

In [378]:
y_pred_wfn = model_logi.predict(test_DTM_WFN)

In [381]:
y_pred_wfn

array(['1', '1', '1', ..., '2', '1', '1'], dtype='<U1')

In [387]:
metrics.accuracy_score(y_test_wfn,y_pred_wfn)

0.8462887200467563

In [391]:
filename = 'paycdClassifer.pkl'  ## pickle.dump(noun_text,open(filename,'wb'))## writnig the model to the file
pickle.dump(model_logi,open(filename,'wb'))